In [ ]:
import base64
import pandas as pd
from PIL import Image

def encode_image_to_base64(img, target_size=-1, fmt='JPEG'):
    # if target_size == -1, will not do resizing
    # else, will set the max_size ot (target_size, target_size)
    if img.mode in ('RGBA', 'P'):
        img = img.convert('RGB')
    if target_size > 0:
        img.thumbnail((target_size, target_size))
    img_buffer = io.BytesIO()
    img.save(img_buffer, format=fmt)
    image_data = img_buffer.getvalue()
    ret = base64.b64encode(image_data).decode('utf-8')
    return ret


def encode_image_file_to_base64(image_path, target_size=-1):
    image = Image.open(image_path)
    return encode_image_to_base64(image, target_size=target_size)


def decode_base64_to_image(base64_string, target_size=-1):
    image_data = base64.b64decode(base64_string)
    image = Image.open(io.BytesIO(image_data))
    if image.mode in ('RGBA', 'P'):
        image = image.convert('RGB')
    if target_size > 0:
        image.thumbnail((target_size, target_size))
    return image


def decode_base64_to_image_file(base64_string, image_path, target_size=-1):
    image = decode_base64_to_image(base64_string, target_size=target_size)
    image.save(image_path)

def split_image(img, output_dir):
    width, height = img.size

    # Calculate coordinates for the 4 parts (quadrants)
    mid_width = width // 2
    mid_height = height // 2

    # Define each quadrant's coordinates (left, upper, right, lower)
    quadrants = {
        "top_left": (0, 0, mid_width, mid_height),
        "top_right": (mid_width, 0, width, mid_height),
        "bottom_left": (0, mid_height, mid_width, height),
        "bottom_right": (mid_width, mid_height, width, height),
        "left": (0, 0, mid_width, height),            # Left half
        "right": (mid_width, 0, width, height),       # Right half
        "top": (0, 0, width, mid_height),             # Top half
        "bottom": (0, mid_height, width, height)      # Bottom half
    }



df = pd.read_csv('/home/srikapan/LMUData/RealWorldQA.tsv', sep='\t')
img = df.loc[0, 'img']